# Velocity Kinematics and Statics

## Introduction
In the last notebook, we studied forward kinematics, which relates joint positions to the end-effector configuration. In this notebook, we will study the relationship between joint velocities and end-effector velocities.

### Forward Kinematics
We often represent the end-effector configuration as a transformation matrix and the velocity as a twist. To quickly grasp the main ideas, we'll represent the end-effector configuration using a minimal set of coordinates and the velocity as the time derivative of those coordinates. The forward kinematics can be written as `x(t) = f(θ(t))`, where `x` is a vector of `m` coordinates representing the end-effector configuration and `θ` is an `n`-vector of joint coordinates.

`[x₁(t) x₂(t) ... xₘ(t)]ᵀ = [f₁(θ) f₂(θ) ... fₘ(θ)]ᵀ ∈ ℝᵐ`

### Relationship between Joint Velocities and End-Effector Velocity
To find the relationship between joint velocities and the end-effector velocity, we take the time derivative of the forward kinematics. Applying the chain rule and dropping the dependence on time, we get `x_dot = J * θ_dot`, where `J` is the Jacobian matrix. The Jacobian is an `m`-by-`n` matrix that relates joint velocities (`θ_dot`) to end-effector velocities (`x_dot`).

$$
\frac{d}{dt} \begin{bmatrix} x_1 \\ x_2 \\ \vdots \\ x_m \end{bmatrix} = \begin{bmatrix} \frac{\partial f_1}{\partial \theta_1} & \frac{\partial f_1}{\partial \theta_2} & \cdots & \frac{\partial f_1}{\partial \theta_n} \\ \frac{\partial f_2}{\partial \theta_1} & \frac{\partial f_2}{\partial \theta_2} & \cdots & \frac{\partial f_2}{\partial \theta_n} \\ \vdots & \vdots & \ddots & \vdots \\ \frac{\partial f_m}{\partial \theta_1} & \frac{\partial f_m}{\partial \theta_2} & \cdots & \frac{\partial f_m}{\partial \theta_n} \end{bmatrix} \begin{bmatrix} \dot{\theta}_1 \\ \dot{\theta}_2 \\ \vdots \\ \dot{\theta}_n \end{bmatrix} \in \mathbb{R}^{m \times n}
$$
$$
\dot{x} = \frac{\partial f}{\partial \theta} \dot{\theta} = J \dot{\theta}
$$

### Jacobian for 2R Robot
Let's consider a 2R robot with end-effector coordinates `x_1` and `x_2`. The forward kinematics are given by:
```
x_1 = L1 * cos(θ_1) + L2 * cos(θ_1 + θ_2)
x_2 = L1 * sin(θ_1) + L2 * sin(θ_1 + θ_2)
```
Taking the time derivative, we get:
```
x_dot_1 = -L1 * sin(θ_1) * θ_dot_1 - L2 * sin(θ_1 + θ_2) * (θ_dot_1 + θ_dot_2)
x_dot_2 = L1 * cos(θ_1) * θ_dot_1 + L2 * cos(θ_1 + θ_2) * (θ_dot_1 + θ_dot_2)
```
We can write this in matrix form as:
$$
v_{\text{tip}} = \begin{bmatrix} \dot{x_1} \\ \dot{x_2} \end{bmatrix} = \begin{bmatrix} -L_1 sin(\theta_1) & -L_2 sin(\theta_1 + \theta_2) \\ L_1 cos(\theta_1) & L_2 cos(\theta_1 + \theta_2) \end{bmatrix}  \dot{\theta}_1 + \begin{bmatrix} -L_2 sin(\theta_1 + \theta_2) \\ L_2 cos(\theta_1 + \theta_2) \end{bmatrix} \dot{\theta}_2
$$
`J₁` is the end-effector velocity when only joint 1 rotates at unit speed while joint 2 is constant. `J₂` is the end-effector velocity when only joint 2 rotates at unit speed while joint 1 is constant. Plotting `J₁` and `J₂`, we see that they form a basis for the space of linear velocities of the end-effector.
$$
v_{\text{tip}} = J_1(\theta) \dot{\theta}_1 + J_2(\theta) \dot{\theta}_2
$$


### Jacobian Matrix `J`
We can combine `J₁` and `J₂` to form the Jacobian matrix `J`. `x_dot = J * θ_dot`, where `J = [J₁ J₂]`. The end-effector velocity is a linear combination of `J₁` and `J₂`, with coefficients equal to the joint velocities. `J` is an `m`-by-`n` matrix, where `m` is the number of end-effector coordinates and `n` is the number of joint coordinates.

$$
v_{\text{tip}} = [J_1(\theta) J_2(\theta)] \begin{bmatrix} \dot{\theta}_1 \\ \dot{\theta}_2 \end{bmatrix} = J(\theta) \dot{\theta}
$$

### Singularity
When the dimension of the column space of the Jacobian drops from its maximum value, the robot is at a singularity. At a singularity, the robot loses the ability to move in certain directions due to the alignment of `J₁` and `J₂`. This configuration limits the robot's motion and requires careful consideration for force control.

### Force Analysis using Jacobian
The Jacobian also relates forces at the end-effector to forces and torques at the joints. The relationship is given by `τ = Jᵀ(θ) * f_tip`, where `τ` is the vector of joint torques and forces generated by motors, and `f_tip` is the force applied by the end-effector. By mapping joint torque limits through the Jacobian, we can determine the limits on the end-effector force.

$$
\tau = \text{vector of joint torques/forces}
$$
$$
\text{power} = \dot{\theta}^T \tau = v_{\text{tip}}^T f_{\text{tip}}
$$
$$
\dot{\theta}^T \tau = (J \dot{\theta})^T f_{\text{tip}} = \dot{\theta}^T J^T f_{\text{tip}}
$$
$$
\tau = J^T f_{\text{tip}}
$$


### Manipulability Ellipsoid
Depending on the robot's configuration, the Jacobian can produce different shapes in the end-effector velocity space. The Jacobian can map a circular joint torque limit to an end-effector force ellipsoid, known as the manipulability ellipsoid. The manipulability ellipsoid indicates the robot's ability to apply forces in different directions.


## Space Jacobian
In the previous section, we discussed the space Jacobian, denoted as J_s, for a general open-chain robot with n joints. The space Jacobian maps joint velocities to the end-effector velocity represented by a twist in the space frame {s}. In this section, we will derive the form of the space Jacobian for a specific example: a 5R arm.
![5R_robot.png](images/5R_robot.png)
source : [Modern Robotics](http://hades.mech.northwestern.edu/images/5/5b/5R_robot.png)
### Space Jacobian for a 5R Arm
Let's consider a 5R arm with joint angles given by θ_1 through θ_5. The space Jacobian for this robot is a 6 by 5 matrix.

$$
J_s(\theta) = \begin{bmatrix}
    J_{s1}(θ) & J_{s2}(θ) & J_{s3}(θ) & J_{s4}(θ) & J_{s5}(θ)
\end{bmatrix}
$$
$$
J_s(θ) \in \mathbb{R}^{6×5}
$$

### Deriving the Space Jacobian J_s3
We will focus on the third column of the space Jacobian, denoted as J_s3, which corresponds to the spatial twist when the velocity at joint 3 is 1, and the velocity at all other joints is zero. If all joint angles are zero, then J_s3 is simply the screw axis S3 of joint 3 when the arm is at its zero configuration. However, we need to find the twist corresponding to a unit velocity at joint 3 in an arbitrary configuration, not just at the zero configuration.

### Analyzing Joint Movements
To find the column J_s3, we will move the robot's joints and observe how it affects J_s3. First, we rotate joint 5, and J_s3 remains unaffected as joint 5 is not between joint 3 and the {s} frame. Similarly, when we rotate joint 3 and 4, J_s3 remains unaffected as these joints are also not between joint 3 and the {s} frame.

### Introducing New Frames
Now, when we rotate joint 2, J_s3 changes. To understand why, we will introduce a new frame {s-prime} that is attached to joint 2. The {s-prime} frame is located at the origin of the {s} frame, and its x-axis is aligned with the x-axis of the {s} frame. The y-axis of the {s-prime} frame is parallel to the y-axis of the {s} frame, and the z-axis of the {s-prime} frame is parallel to the z-axis of the {s} frame. The {s-prime} frame is rotated by θ_2 about the z-axis of the {s} frame. 

$$
T_{s-prime} = e^{[S_2]\theta_2}
$$

When rotating joint 1, J_s3 also changes. We will introduce another new frame {s-double-prime} to represent this change.

$$
T_{s-double-prime} = e^{[S_1]\theta_1} e^{[S_2]\theta_2}
$$

### Deriving J_s3 Expression
To find J_s3, we express S3, now corresponding to the screw axis in the {s-double-prime} frame, in the {s} frame. Using the standard rule for changing the reference frame of a twist, we derive the final expression for J_s3.

$$
J_{s3}(θ) =  [Ad_{T_{ss\prime\prime}}] S_3 = [Ad_{e^{[S_1]\theta_1} e^{[S_2]\theta_2}}] S_3
$$
### Generalizing to Other Joints
The same reasoning applies for any joint, not just joint 3 of this 5R robot. Joint positions of joints between the joint and the {s} frame must be taken into account, while joint positions that do not affect the relationship between the joint and the {s} frame can be ignored.

### General Form of Space Jacobian J_s
The general form of the space Jacobian J_s is derived as follows:
- The first column of J_s is just the screw axis S1 when the robot is at its zero configuration. It does not depend on the joint positions.
- Any other column i of J_s is given by the screw axis S_i premultiplied by the transformation that expresses the screw axis in the {s} frame for arbitrary joint positions.
- The space Jacobian J_s is independent of the choice of the end-effector {b} frame.
$$
\text{The space Jacobian } J_s(\theta) \text{ is given by:}
$$
$$
V_s = J_s(\theta) \dot{\theta}
$$
$$
\text{where } J_s(\theta) =  [J_{s1}(\theta) \ J_{s2}(\theta) \ \dots \ J_{sn}(\theta)] \in \mathbb{R}^{6×n}
$$
$$
\text{with } J_{s1} = S_1, \text{ and } J_{si}(\theta) = [Ad_{T_{e^{[S_1]\theta_1} \dots e^{[S_{i-1}]\theta_{i-1}}}}] S_i \text{ for } i = 2, \dots, n
$$


## Body Jacobian
In the previous section, we learned how to construct the space Jacobian J_s for a 5R arm, which maps joint velocities to the end-effector twist represented in the space frame {s}. In this section, we will construct the body Jacobian J_b for the same 5R arm. The body Jacobian transforms joint velocities to the end-effector twist represented in the end-effector frame {b}.

$$
v_b = J_b(\theta) \dot{\theta}
$$

$$
J_b(\theta) \in \mathbb{R}^{6 \times n}
$$

### Body Jacobian for a 5R Arm
The body Jacobian J_b has five columns, one for each joint, and in this video, we will focus on J_b3, the third column, corresponding to the end-effector twist when joint 3 moves with unit velocity.

$$
J_b(\theta) = \begin{bmatrix} J_b1 & J_b2 & J_b3 & J_b4 & J_b5 \end{bmatrix}
$$

### Deriving J_b3 Expression
To derive J_b3, we first set all joint angles equal to zero. At this configuration, J_b3 is just the screw axis B_3 of joint 3 expressed in the {b} frame when the arm is at its zero configuration. Now, when we rotate joint 1 and joint 2, J_b3 remains unaffected as their motion does not change the relationship between joint 3 and the {b} frame.

### Defining New Frames
Next, we rotate joint 4, and J_b3 changes as the configuration of joint 3 relative to the {b} frame moves. To handle this change, we define a new frame {b-double-prime} to be the {b} frame before joint 4 is rotated, and the frame {b-prime} to be the {b} frame after joint 4 is rotated. The relationship between these frames is given by the transformation T_b-double-prime_b-prime, which involves the screw axis B_4 and the joint angle θ_4.

$$
T_{b''b'} = e^{[B_4] \theta_4}
$$

### Finding J_b3
Finally, we rotate joint 5, giving us the final end-effector frame {b}, obtained by rotating the frame {b-prime} about the joint 5 screw axis by θ_5. To find J_b3, we use the rule for changing the frame of reference of a twist, and the final expression for J_b3 depends on the screw axes for joints 3, 4, and 5, as well as the joint angles.

$$
T_{b''b} = e^{[B_4] \theta_4} e^{[B_5] \theta_5}
$$
$$
T_{bb''} = (T_{b''b})^{-1} = e^{-[B_5] \theta_5} e^{-[B_4] \theta_4}
$$
$$
J_{b3}(\theta) = [Ad_{T_{bb''}}] B_3
$$

### Generalizing to Other Joints
The same reasoning applies for any joint, so we can generalize to the definition of the body Jacobian J_b. The last column of J_b is just the screw axis B_n when the robot is at its zero configuration, and it does not depend on the joint positions. Any other column i of J_b is given by the screw axis B_i premultiplied by the transformation that expresses the screw axis in the {b} frame for arbitrary joint positions.

$$
\text{The body Jacobian } J_b(\theta) \text{ is given by:}
$$
$$
v_b = J_b(\theta) \dot{\theta}
$$
$$
\text{where } J_b(\theta) =  \begin{bmatrix} J_b1 & J_b2 & \cdots & J_bn \end{bmatrix} \in \mathbb{R}^{6 \times n}
$$
$$
\text{with } J_bn = B_n \text{ and } J_{bi}(θ) = [Ad_{T_{bb''}}] B_i \text{ for } i = 1, 2, \cdots, n-1
$$

### Relationship Between J_s and J_b
Since each column of a Jacobian is a twist, we can use the rule for representing a twist in a different frame to translate between the space Jacobian J_s and the body Jacobian J_b. J_b is obtained from J_s by the matrix adjoint of T_bs, and J_s is obtained from J_b by the matrix adjoint of T_sb.

$$
J_b(\theta) = [Ad_{T_{bs}}] J_s(\theta)
$$
$$
J_s(\theta) = [Ad_{T_{sb}}] J_b(\theta)
$$


## Statics of Open Chains
In this section, we explore the relationship between wrenches applied to a robot's end-effector and the resulting joint torques and forces required to maintain the robot's trajectory. We will see how the Jacobian matrix plays a crucial role in this relationship.

### Wrenches and Torques
Consider a 6R robot with a frame {b} at the hand. As the robot's joints move according to a trajectory θ(t), the hand moves along a yellow path. Assume the hand is moving in free space, applying no forces to the environment. When we study the inverse dynamics of a robot, we will learn how the trajectory θ(t) can be translated into the torques required to move the robot along that trajectory, denoted as τ-motion(t).

### Disturbance Wrench
Now, let's consider a particular time instant t, and at this instant, assume that someone applies a wrench to the hand. This wrench is represented as -F_b and consists of three angular moments and three linear forces expressed in the {b} frame. If we want the robot to continue tracking the planned trajectory despite this disturbance wrench, the robot's motors must generate a wrench F_b to balance the disturbance wrench. Thus, the joint torques should be τ-motion(t) plus τ, where we need to know how F_b relates to τ.

$$
-F_b = \begin{bmatrix} -m_{bx} // -m_{by} // -m_{bz} // -f_{bx} // -f_{by} // -f_{bz} \end{bmatrix}
$$

### Relationship Between Wrench and Torque
To find this relationship, we use the principle from physics that force times velocity is power. In the {b} frame, the wrench F_b created by the motors multiplies the twist V_b to get the mechanical power produced or consumed at the hand. This power comes from the motors, which is represented as the joint torques dotted with the joint velocities. By applying the identity J_bθ-dot = V_b and recognizing that the equality must hold for all θ-dot, we derive the equation :

$$
\tau = J_b^T(\theta)F_b
$$


### Generalization
The same derivation holds for wrenches and Jacobians expressed in the space frame {s}. Thus, we can generalize the result: 

to resist a wrench -F applied to the end-effector at a configuration θ, the joint torques and forces τ must be J of θ transposed times F. This relationship holds regardless of the frame in which the Jacobian and wrench are expressed.

$$
\tau = J_*^T(\theta)F_*
$$

This relationship is particularly useful in force control of a robot. If we want the end-effector to apply a wrench F to the environment, we can use this formula to calculate the joint forces and torques τ needed to achieve that wrench.